# Get rdf of all the polymers

In [ ]:
import numpy as np
import copy
import matplotlib.pyplot as plt
from scipy import optimize
from matplotlib.colors import ListedColormap

import psutil
import os
import gc

np.random.seed([170786])

In [ ]:
def read_traj(route,method_):
    file = open(f"stretch{temperature}/{route}","r")
    info_r1 = file.readlines()
    file.close()
    
    
    if method_==0:
        info_s1 = []
        for i in range (len(info_r1)):
            info_s1.append(info_r1[i].split())
        info_r1 = []
        
        
    print("---------------------------------------------------------")
    info = psutil.virtual_memory()
#     print(psutil.Process(os.getpid()).memory_info().rss)
#     print(info.total)
    print(info.percent)
    
#     print(psutil.cpu_count())


    Natom_s1 = int(info_s1[3][0])
    index_s1 = []
    for i in range (len(info_s1)):
        if info_s1[i] == ['ITEM:', 'TIMESTEP']:
            index_s1.append(i)

    splited_trj1 = []
    for j in range (len(index_s1)):
        frame = np.zeros((Natom_s1,5))
        for i in range (Natom_s1):
            frame[i,0] = int(info_s1[index_s1[j]+i+9][0])
            frame[i,1] = float(info_s1[index_s1[j]+i+9][1])
            frame[i,2] = float(info_s1[index_s1[j]+i+9][2])
            frame[i,3] = float(info_s1[index_s1[j]+i+9][3])
            frame[i,4] = float(info_s1[index_s1[j]+i+9][4])
        splited_trj1.append([frame,info_s1[index_s1[j]+5:index_s1[j]+8]])
    
    return splited_trj1

In [ ]:
def mass_center1(pos1,bx,by,bz,bottom_x,bottom_y,bottom_z):
    c = np.zeros((1,3))
    
    c[0,0] = (pos1[0,0]-bottom_x)-bx*np.floor((pos1[0,0]-bottom_x)/bx)
    c[0,1] = (pos1[0,1]-bottom_y)-by*np.floor((pos1[0,1]-bottom_y)/by)
    c[0,2] = (pos1[0,2]-bottom_z)-bz*np.floor((pos1[0,2]-bottom_z)/bz)
    
    return c

In [ ]:
def mass_center2(pos1,pos2,bx,by,bz,bottom_x,bottom_y,bottom_z):
    c = np.zeros((1,3))
    
    c[0,0] = (pos1[0]+pos2[0])/2
    c[0,1] = (pos1[1]+pos2[1])/2
    c[0,2] = (pos1[2]+pos2[2])/2
    
    return mass_center1(c,bx,by,bz,bottom_x,bottom_y,bottom_z)[0]

In [ ]:
def mass_center3(pos1,pos2,pos3,bx,by,bz,bottom_x,bottom_y,bottom_z):
    c = np.zeros((1,3))
    
    c[0,0] = (pos1[0]+pos2[0]+pos3[0])/3
    c[0,1] = (pos1[1]+pos2[1]+pos3[1])/3
    c[0,2] = (pos1[2]+pos2[2]+pos3[2])/3
    
    return mass_center1(c,bx,by,bz,bottom_x,bottom_y,bottom_z)[0]

In [ ]:
def extract_information(traj,range_min,range_max):
    ## distribution
    
    N_index = []
    for i in range (len(traj[0][0])):
        if traj[0][0][i][1] == 14.007:
            N_index.append(i)
    
    A_center = []
    D_center = []
    
    for target in range (range_min,range_max,10):
        
        bx_low = float(traj[target][1][0][0])
        by_low = float(traj[target][1][1][0])
        bz_low = float(traj[target][1][2][0])
        
        boundary_x = float(traj[target][1][0][1])-bx_low
        boundary_y = float(traj[target][1][1][1])-by_low
        boundary_z = float(traj[target][1][2][1])-bz_low
        
        A_info = []
        D_info = []
        
        for i in range (int(len(N_index)/2)):
            A_info.append(mass_center2(traj[target][0][N_index[2*i+1]-4][2:],traj[target][0][N_index[2*i+1]-1][2:],boundary_x,boundary_y,boundary_z,bx_low,by_low,bz_low))
            D_info.append(mass_center2(traj[target][0][N_index[2*i+1]][2:],traj[target][0][N_index[2*i+1]+7][2:],boundary_x,boundary_y,boundary_z,bx_low,by_low,bz_low))
            
        A_center.append([A_info,boundary_x,boundary_y,boundary_z])
        D_center.append([D_info,boundary_x,boundary_y,boundary_z])
        
    return A_center, D_center

In [ ]:
def wrap_coordiante(pos1,bx,by,bz):
   
    new_pos = np.zeros((1,3))
    
    new_pos[0,0] = pos1[0]-bx*np.floor(pos1[0]/bx)
    new_pos[0,1] = pos1[1]-by*np.floor(pos1[1]/by)
    new_pos[0,2] = pos1[2]-bz*np.floor(pos1[2]/bz)

    return new_pos

In [ ]:
def center_distance(n1,n2,bx,by,bz):
    
    dis = wrap_distance(n1,n2,bx,by,bz)
    
    return dis

In [ ]:
def wrap_distance(n1,n2,bx,by,bz):
    relevant_pos = np.zeros((1,3))
    new_coordiantes = np.zeros((1,3))
    
    relevant_pos[0,0] = n2[0,0]-n1[0,0]
    relevant_pos[0,1] = n2[0,1]-n1[0,1]
    relevant_pos[0,2] = n2[0,2]-n1[0,2]
    
    new_coordiantes[0,0] = relevant_pos[0,0]-bx*np.round(relevant_pos[0,0]/bx)
    new_coordiantes[0,1] = relevant_pos[0,1]-by*np.round(relevant_pos[0,1]/by)
    new_coordiantes[0,2] = relevant_pos[0,2]-bz*np.round(relevant_pos[0,2]/bz)
    
    return np.sqrt(new_coordiantes[0,0]**2+new_coordiantes[0,1]**2+new_coordiantes[0,2]**2)

In [ ]:
def gr_1st(new_a,b_dis,bx,by,bz,sep):
    
    count = np.zeros((len(sep)-1,2))
    for j in range (len(sep)-1):
        count[j,0] = sep[j+1]
    
    info = []
    for i in range (len(b_dis)):
        new_b = wrap_coordiante(b_dis[i],bx,by,bz)
        dis = center_distance(new_a,new_b,bx,by,bz)
        if dis <= up_limit and dis!= 0:
            info.append(dis)
    
#     rho0 = len(info)/(4/3*np.pi*(up_limit**3))
    rho0 = len(b_dis)/(bx*by*bz)
#     print("samples = %d"%(len(info))+", "+"density = %.4f"%(len(info)/(4/3*np.pi*(up_limit**3))))
#     print("density = %.4f"%(len(b_dis)/(bx*by*bz)))
#     print("------------")
    for p in range (len(info)):
        for q in range (len(count)):
            if info[p]>sep[q] and info[p]<=sep[q+1]:
                count[q,1] += 1/rho0

    return count

In [ ]:
def gr(a_dis,b_dis,up_limit,interval):
    
    bins = int(np.ceil(up_limit/interval))
    sep = np.linspace(0,1,bins+1)

    
    for j in range (bins):
        sep[j+1] = (j+1)*interval
    sep[bins] = up_limit
    
    
    gr_time = []
    for target in range (len(a_dis)):
        
        gr_all = np.zeros((bins,2))
        for j in range (bins):
            gr_all[j,0] = sep[j+1]
        
        info = []
        bx = a_dis[target][1]
        by = a_dis[target][2]
        bz = a_dis[target][3]
        
        for a in range (len(a_dis[target][0])):
            new_a = wrap_coordiante(a_dis[target][0][a],bx,by,bz)
            gr_a = gr_1st(new_a,b_dis[target][0],bx,by,bz,sep)
            
            for p in range (len(gr_a)):
                gr_all[p,1] += gr_a[p,1]
            
        for q in range (len(gr_a)):
            gr_all[q,1] = gr_all[q,1]/len(a_dis[target][0])/(4/3*np.pi*(sep[q+1]**3-sep[q]**3))
        
        gr_time.append(gr_all)
    
    return gr_time

In [ ]:
def gauss1D(x,mu=0,sigma=1):
    return np.exp(-0.5*(x-mu)**2/sigma/sigma)/np.sqrt(2.*np.pi)/sigma

def gr_smooth(a_dis,b_dis,up_limit,interval):
    
    NRmax = int(np.ceil(up_limit/interval))+1
    
    bins = np.linspace(0,up_limit,NRmax,endpoint=True)
    radii = np.zeros(shape=(NRmax-1,),dtype="f8")
    volumes = np.zeros(shape=(NRmax-1,),dtype="f8")
    
    for i in range(NRmax-1):
        radii[i] = (bins[i]+bins[i+1])/2. # middle point between two bins
        volumes[i] = 4/3*np.pi*(bins[i+1]**3-bins[i]**3)
    
    gr_time = []
    for target in range (len(a_dis)):
        
        hist = np.zeros(shape=(NRmax-1,),dtype="f8")
        bx = a_dis[target][1]
        by = a_dis[target][2]
        bz = a_dis[target][3]
        
    
        for a in range (len(a_dis[target][0])):        
            new_a = wrap_coordiante(a_dis[target][0][a],bx,by,bz)
            
            for b in range (len(b_dis[target][0])):
                new_b = wrap_coordiante(b_dis[target][0][b],bx,by,bz)
                dis = center_distance(new_a,new_b,bx,by,bz)
                if dis<=up_limit and dis!=0:
                    hist += gauss1D(radii,dis,0.6)*up_limit/NRmax
         
        gr_all = np.zeros((len(bins)-1,2))
        for j in range (len(bins)-1):
            gr_all[j,0] = radii[j]
            gr_all[j,1] = hist[j]/volumes[j]/len(a_dis[target][0])/(len(b_dis[target][0])/(bx*by*bz))
        
        gr_time.append(gr_all)
    
    return gr_time

In [ ]:
def write_data(data_AA,data_DD,data_AA_smooth,data_DD_smooth,name,strain):
    
    def average(data):
        info_ = np.zeros((len(data[0]),2))
        
        for r in range (len(data[0])):
            info_[r,0] = data[0][r,0]
            for t in range (len(data)):
                info_[r,1] += data[t][r,1]/len(data)
        return info_
            
    info_aa = average(data_AA)
    info_dd = average(data_DD)
    info_aas = average(data_AA_smooth)
    info_dds = average(data_DD_smooth)
    
    file = open(f"{name}_{strain}.txt","w")
    file.write("distance      AA      DD      AA(s)      DD(s)")
    file.write("\n")
    
    for i in range (len(info_aa)):
        file.write("%.2f"%(info_aa[i,0])+"      ")
        file.write("%.4f"%(info_aa[i,1])+"     ")
        file.write("%.4f"%(info_dd[i,1])+"     ")
        file.write("%.4f"%(info_aas[i,1])+"     ")
        file.write("%.4f"%(info_dds[i,1])+"     ")
        file.write("\n")
    file.close()

In [ ]:
def have_a_look(data_ori,data_smooth,name,type_):
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)

    def average(data):
        info_ = np.zeros((len(data[0]),2))

        for r in range (len(data[0])):
            info_[r,0] = data[0][r,0]
            for t in range (len(data)):
                info_[r,1] += data[t][r,1]/len(data)
        return info_

    test1 = average(data_ori)
    test2 = average(data_smooth)

    ax.plot(test1[:-10,0],test1[:-10,1],color="black",label='ori')
    ax.plot(test2[:-10,0],test2[:-10,1],color="red",label='smooth')
    
    ax.set_xlabel("r ($\AA$)",fontsize=20)
    ax.set_ylabel("$g_{r}$",fontsize=20)
    ax.set_title(f"{name} {type_}",fontsize=20)
    
    plt.legend()
    plt.show()

# M

In [ ]:
name = "pdkcm"
sname = "m"
bname="PDKCM"
temperature = 1.2

up_limit = 30
interval = 0.3

splited_m_0 = read_traj("s0-ta.trj",0)
A_m_0, D_m_0 = extract_information(splited_m_0,0,len(splited_m_0))

splited_m_50 = read_traj("s13_ta.trj",0)
A_m_50, D_m_50 = extract_information(splited_m_50,0,len(splited_m_50))

splited_m_100 = read_traj("s27_ta.trj",0)
A_m_100, D_m_100 = extract_information(splited_m_100,0,len(splited_m_100))

### 0%

In [ ]:
# gr_m_0_ad = gr(A_m_0,D_m_0,up_limit,interval)
# gr_m_0_ad_s = gr_smooth(A_m_0,D_m_0,up_limit,interval)

gr_m_0_aa = gr(A_m_0,A_m_0,up_limit,interval)
gr_m_0_aa_s = gr_smooth(A_m_0,A_m_0,up_limit,interval)

gr_m_0_dd = gr(D_m_0,D_m_0,up_limit,interval)
gr_m_0_dd_s = gr_smooth(D_m_0,D_m_0,up_limit,interval)

# gr_m_0_da = gr(D_m_0,A_m_0,up_limit,interval)
# gr_m_0_da_s = gr_smooth(D_m_0,A_m_0,up_limit,interval)

### 50%

In [ ]:
# # gr_m_50_ad = gr(A_m_50,D_m_50,up_limit,interval)
# # gr_m_50_ad_s = gr_smooth(A_m_50,D_m_50,up_limit,interval)

gr_m_50_aa = gr(A_m_50,A_m_50,up_limit,interval)
gr_m_50_aa_s = gr_smooth(A_m_50,A_m_50,up_limit,interval)

gr_m_50_dd = gr(D_m_50,D_m_50,up_limit,interval)
gr_m_50_dd_s = gr_smooth(D_m_50,D_m_50,up_limit,interval)

# # gr_m_50_da = gr(D_m_50,A_m_50,up_limit,interval)
# # gr_m_50_da_s = gr_smooth(D_m_50,A_m_50,up_limit,interval)

### 100%

In [ ]:
# # gr_m_100_ad = gr(A_m_100,D_m_100,up_limit,interval)
# # gr_m_100_ad_s = gr_smooth(A_m_100,D_m_100,up_limit,interval)

gr_m_100_aa = gr(A_m_100,A_m_100,up_limit,interval)
gr_m_100_aa_s = gr_smooth(A_m_100,A_m_100,up_limit,interval)

gr_m_100_dd = gr(D_m_100,D_m_100,up_limit,interval)
gr_m_100_dd_s = gr_smooth(D_m_100,D_m_100,up_limit,interval)

# # gr_m_100_da = gr(D_m_100,A_m_100,up_limit,interval)
# # gr_m_100_da_s = gr_smooth(D_m_100,A_m_100,up_limit,interval)

In [ ]:
have_a_look(gr_m_0_aa,gr_m_0_aa_s,"PDKCM","0% AA")
have_a_look(gr_m_0_dd,gr_m_0_dd_s,"PDKCM","0% DD")

have_a_look(gr_m_50_aa,gr_m_50_aa_s,"PDKCM","50% AA")
have_a_look(gr_m_50_dd,gr_m_50_dd_s,"PDKCM","50% DD")

have_a_look(gr_m_100_aa,gr_m_100_aa_s,"PDKCM","100% AA")
have_a_look(gr_m_100_dd,gr_m_100_dd_s,"PDKCM","100% DD")

In [ ]:
write_data(gr_m_0_aa,gr_m_0_dd,gr_m_0_aa_s,gr_m_0_dd_s,"M",0)
write_data(gr_m_50_aa,gr_m_50_dd,gr_m_50_aa_s,gr_m_50_dd_s,"M",50)
write_data(gr_m_100_aa,gr_m_100_dd,gr_m_100_aa_s,gr_m_100_dd_s,"M",100)

splited_m_0 = []
splited_m_50 = []
splited_m_100 = []

# P

In [ ]:
name = "pdkcp"
sname = "p"
bname="PDKCP"
temperature = 1.2

up_limit = 30
interval = 0.3

splited_p_0 = read_traj(f"s0_ta.trj",0)
A_p_0, D_p_0 = extract_information(splited_p_0,0,len(splited_p_0))

splited_p_50 = read_traj(f"s17_ta.trj",0)
A_p_50, D_p_50 = extract_information(splited_p_50,0,len(splited_p_50))

splited_p_100 = read_traj(f"s27_ta.trj",0)
A_p_100, D_p_100 = extract_information(splited_p_100,0,len(splited_p_100))

### 0%

In [ ]:
# gr_p_0_ad = gr(A_p_0,D_p_0,up_limit,interval)
# gr_p_0_ad_s = gr_smooth(A_p_0,D_p_0,up_limit,interval)

gr_p_0_aa = gr(A_p_0,A_p_0,up_limit,interval)
gr_p_0_aa_s = gr_smooth(A_p_0,A_p_0,up_limit,interval)

gr_p_0_dd = gr(D_p_0,D_p_0,up_limit,interval)
gr_p_0_dd_s = gr_smooth(D_p_0,D_p_0,up_limit,interval)

# gr_p_0_da = gr(D_p_0,A_p_0,up_limit,interval)
# gr_p_0_da_s = gr_smooth(D_p_0,A_p_0,up_limit,interval)

### 50%

In [ ]:
# # gr_p_50_ad = gr(A_p_50,D_p_50,up_limit,interval)
# # gr_p_50_ad_s = gr_smooth(A_p_50,D_p_50,up_limit,interval)

gr_p_50_aa = gr(A_p_50,A_p_50,up_limit,interval)
gr_p_50_aa_s = gr_smooth(A_p_50,A_p_50,up_limit,interval)

gr_p_50_dd = gr(D_p_50,D_p_50,up_limit,interval)
gr_p_50_dd_s = gr_smooth(D_p_50,D_p_50,up_limit,interval)

# # gr_p_50_da = gr(D_p_50,A_p_50,up_limit,interval)
# # gr_p_50_da_s = gr_smooth(D_p_50,A_p_50,up_limit,interval)

### 100%

In [ ]:
# # gr_p_100_ad = gr(A_p_100,D_p_100,up_limit,interval)
# # gr_p_100_ad_s = gr_smooth(A_p_100,D_p_100,up_limit,interval)

gr_p_100_aa = gr(A_p_100,A_p_100,up_limit,interval)
gr_p_100_aa_s = gr_smooth(A_p_100,A_p_100,up_limit,interval)

gr_p_100_dd = gr(D_p_100,D_p_100,up_limit,interval)
gr_p_100_dd_s = gr_smooth(D_p_100,D_p_100,up_limit,interval)

# # gr_p_100_da = gr(D_p_100,A_p_100,up_limit,interval)
# # gr_p_100_da_s = gr_smooth(D_p_100,A_p_100,up_limit,interval)

In [ ]:
have_a_look(gr_p_0_aa,gr_p_0_aa_s,"PDKCP","0% AA")
have_a_look(gr_p_0_dd,gr_p_0_dd_s,"PDKCP","0% DD")

have_a_look(gr_p_50_aa,gr_p_50_aa_s,"PDKCP","50% AA")
have_a_look(gr_p_50_dd,gr_p_50_dd_s,"PDKCP","50% DD")

have_a_look(gr_p_100_aa,gr_p_100_aa_s,"PDKCP","100% AA")
have_a_look(gr_p_100_dd,gr_p_100_dd_s,"PDKCP","100% DD")

In [ ]:
write_data(gr_p_0_aa,gr_p_0_dd,gr_p_0_aa_s,gr_p_0_dd_s,"P",0)
write_data(gr_p_50_aa,gr_p_50_dd,gr_p_50_aa_s,gr_p_50_dd_s,"P",50)
write_data(gr_p_100_aa,gr_p_100_dd,gr_p_100_aa_s,gr_p_100_dd_s,"P",100)

splited_p_0 = []
splited_p_50 = []
splited_p_100 = []

# H

In [ ]:
name = "pdkch"
sname = "h"
bname="PDKCH"
temperature = 1.2

up_limit = 30
interval = 0.3

splited_h_0 = read_traj(f"s0_ta.trj",0)
A_h_0, D_h_0 = extract_information(splited_h_0,0,len(splited_h_0))
splited_h_0_2 = read_traj(f"s0_ta-2.trj",0)
A_h_0_2, D_h_0_2 = extract_information(splited_h_0_2,0,len(splited_h_0_2))
for i in range (1,len(A_h_0_2)):
    A_h_0.append(A_h_0_2[i])
for i in range (1,len(D_h_0_2)):
    D_h_0.append(D_h_0_2[i])

splited_h_50 = read_traj(f"s17_ta.trj",0)
A_h_50, D_h_50 = extract_information(splited_h_50,0,len(splited_h_50))
splited_h_50_2 = read_traj(f"s17_ta-2.trj",0)
A_h_50_2, D_h_50_2 = extract_information(splited_h_50_2,0,len(splited_h_50_2))
for i in range (1,len(A_h_50_2)):
    A_h_50.append(A_h_50_2[i])
for i in range (1,len(D_h_50_2)):
    D_h_50.append(D_h_50_2[i])

splited_h_100 = read_traj(f"s27_ta.trj",0)
A_h_100, D_h_100 = extract_information(splited_h_100,0,len(splited_h_100))

In [ ]:
splited_h_100 = read_traj(f"s27_ta.trj",0)
A_h_100, D_h_100 = extract_information(splited_h_100,0,len(splited_h_100))

## 0%

In [ ]:
# gr_h_0_ad = gr(A_h_0,D_h_0,up_limit,interval)
# gr_h_0_ad_s = gr_smooth(A_h_0,D_h_0,up_limit,interval)

gr_h_0_aa = gr(A_h_0,A_h_0,up_limit,interval)
gr_h_0_aa_s = gr_smooth(A_h_0,A_h_0,up_limit,interval)

gr_h_0_dd = gr(D_h_0,D_h_0,up_limit,interval)
gr_h_0_dd_s = gr_smooth(D_h_0,D_h_0,up_limit,interval)

# gr_h_0_da = gr(D_h_0,A_h_0,up_limit,interval)
# gr_h_0_da_s = gr_smooth(D_h_0,A_h_0,up_limit,interval)

### 50%

In [ ]:
# # gr_h_50_ad = gr(A_h_50,D_h_50,up_limit,interval)
# # gr_h_50_ad_s = gr_smooth(A_h_50,D_h_50,up_limit,interval)

gr_h_50_aa = gr(A_h_50,A_h_50,up_limit,interval)
gr_h_50_aa_s = gr_smooth(A_h_50,A_h_50,up_limit,interval)

gr_h_50_dd = gr(D_h_50,D_h_50,up_limit,interval)
gr_h_50_dd_s = gr_smooth(D_h_50,D_h_50,up_limit,interval)

# # gr_h_50_da = gr(D_h_50,A_h_50,up_limit,interval)
# # gr_h_50_da_s = gr_smooth(D_h_50,A_h_50,up_limit,interval)

### 100%

In [ ]:
# # gr_h_100_ad = gr(A_h_100,D_h_100,up_limit,interval)
# # gr_h_100_ad_s = gr_smooth(A_h_100,D_h_100,up_limit,interval)

gr_h_100_aa = gr(A_h_100,A_h_100,up_limit,interval)
gr_h_100_aa_s = gr_smooth(A_h_100,A_h_100,up_limit,interval)

gr_h_100_dd = gr(D_h_100,D_h_100,up_limit,interval)
gr_h_100_dd_s = gr_smooth(D_h_100,D_h_100,up_limit,interval)

# # gr_h_100_da = gr(D_h_100,A_h_100,up_limit,interval)
# # gr_h_100_da_s = gr_smooth(D_h_100,A_h_100,up_limit,interval)

In [ ]:
have_a_look(gr_h_0_aa,gr_h_0_aa_s,"PDKCH","0% AA")
have_a_look(gr_h_0_dd,gr_h_0_dd_s,"PDKCH","0% DD")

have_a_look(gr_h_50_aa,gr_h_50_aa_s,"PDKCH","50% AA")
have_a_look(gr_h_50_dd,gr_h_50_dd_s,"PDKCH","50% DD")

have_a_look(gr_h_100_aa,gr_h_100_aa_s,"PDKCH","100% AA")
have_a_look(gr_h_100_dd,gr_h_100_dd_s,"PDKCH","100% DD")

In [ ]:
write_data(gr_h_0_aa,gr_h_0_dd,gr_h_0_aa_s,gr_h_0_dd_s,"H",0)
write_data(gr_h_50_aa,gr_h_50_dd,gr_h_50_aa_s,gr_h_50_dd_s,"H",50)
write_data(gr_h_100_aa,gr_h_100_dd,gr_h_100_aa_s,gr_h_100_dd_s,"H",100)

splited_h_0 = []
splited_h_50 = []
splited_h_100 = []

# D

In [ ]:
name = "pdkcd"
sname = "d"
bname="PDKCD"
temperature = 1.2

up_limit = 30
interval = 0.3

splited_d_0 = read_traj(f"s0_ta-1.trj",0)
A_d_0, D_d_0 = extract_information(splited_d_0,0,len(splited_d_0))
splited_d_0_2 = read_traj(f"s0_ta-2.trj",0)
A_d_0_2, D_d_0_2 = extract_information(splited_d_0_2,0,len(splited_d_0_2))

for i in range (1,len(A_d_0_2)):
    A_d_0.append(A_d_0_2[i])
for i in range (1,len(D_d_0_2)):
    D_d_0.append(D_d_0_2[i])


splited_d_50 = read_traj(f"s10_ta.trj",0)
A_d_50, D_d_50 = extract_information(splited_d_50,0,len(splited_d_50))

splited_d_100 = read_traj(f"s20_ta.trj",0)
A_d_100, D_d_100 = extract_information(splited_d_100,0,len(splited_d_100))

### 0%

In [ ]:
# gr_d_0_ad = gr(A_d_0,D_d_0,up_limit,interval)
# gr_d_0_ad_s = gr_smooth(A_d_0,D_d_0,up_limit,interval)

gr_d_0_aa = gr(A_d_0,A_d_0,up_limit,interval)
gr_d_0_aa_s = gr_smooth(A_d_0,A_d_0,up_limit,interval)

gr_d_0_dd = gr(D_d_0,D_d_0,up_limit,interval)
gr_d_0_dd_s = gr_smooth(D_d_0,D_d_0,up_limit,interval)

# gr_d_0_da = gr(D_d_0,A_d_0,up_limit,interval)
# gr_d_0_da_s = gr_smooth(D_d_0,A_d_0,up_limit,interval)

### 50%

In [ ]:
# # gr_d_50_ad = gr(A_d_50,D_d_50,up_limit,interval)
# # gr_d_50_ad_s = gr_smooth(A_d_50,D_d_50,up_limit,interval)

gr_d_50_aa = gr(A_d_50,A_d_50,up_limit,interval)
gr_d_50_aa_s = gr_smooth(A_d_50,A_d_50,up_limit,interval)

gr_d_50_dd = gr(D_d_50,D_d_50,up_limit,interval)
gr_d_50_dd_s = gr_smooth(D_d_50,D_d_50,up_limit,interval)

# # gr_d_50_da = gr(D_d_50,A_d_50,up_limit,interval)
# # gr_d_50_da_s = gr_smooth(D_d_50,A_d_50,up_limit,interval)

### 100%

In [ ]:
# # gr_d_100_ad = gr(A_d_100,D_d_100,up_limit,interval)
# # gr_d_100_ad_s = gr_smooth(A_d_100,D_d_100,up_limit,interval)

gr_d_100_aa = gr(A_d_100,A_d_100,up_limit,interval)
gr_d_100_aa_s = gr_smooth(A_d_100,A_d_100,up_limit,interval)

gr_d_100_dd = gr(D_d_100,D_d_100,up_limit,interval)
gr_d_100_dd_s = gr_smooth(D_d_100,D_d_100,up_limit,interval)

# # gr_d_100_da = gr(D_d_100,A_d_100,up_limit,interval)
# # gr_d_100_da_s = gr_smooth(D_d_100,A_d_100,up_limit,interval)

In [ ]:
have_a_look(gr_d_0_aa,gr_d_0_aa_s,"PDKCD","0% AA")
have_a_look(gr_d_0_dd,gr_d_0_dd_s,"PDKCD","0% AA")

have_a_look(gr_d_50_aa,gr_d_50_aa_s,"PDKCD","50% AA")
have_a_look(gr_d_50_dd,gr_d_50_dd_s,"PDKCD","50% DD")

have_a_look(gr_d_100_aa,gr_d_100_aa_s,"PDKCD","100% AA")
have_a_look(gr_d_100_dd,gr_d_100_dd_s,"PDKCD","100% DD")

In [ ]:
write_data(gr_d_0_aa,gr_d_0_dd,gr_d_0_aa_s,gr_d_0_dd_s,"D",0)
write_data(gr_d_50_aa,gr_d_50_dd,gr_d_50_aa_s,gr_d_50_dd_s,"D",50)
write_data(gr_d_100_aa,gr_d_100_dd,gr_d_100_aa_s,gr_d_100_dd_s,"D",100)

splited_d_0 = []
splited_d_50 = []
splited_d_100 = []

# TEST

In [ ]:
# def comparison_strain(data_0,data_50,data_100,bname,type_):
#     fig = plt.figure(figsize=(8,6))
#     ax = fig.add_subplot(111)
    
#     def average(data):
#         info_ = np.zeros((len(data[0]),2))
        
#         for r in range (len(data[0])):
#             info_[r,0] = data[0][r,0]
#             for t in range (len(data)):
#                 info_[r,1] += data[t][r,1]/len(data)
#         return info_
            
#     info_0 = average(data_0)
#     info_50 = average(data_0)
#     info_100 = average(data_0)
    
    
#     ax.plot(info_0[:-10,0],info_0[:-10,1],color="black",label='0% strain')
#     ax.plot(info_50[:-10,0],info_50[:-10,1],color="red",label='50% strain')
#     ax.plot(info_100[:-10,0],info_100[:-10,1],color="green",label='100% strain')
    
#     if type_=="AD":
#         ax.set_xlabel("$r_{AD}$ ($\AA$)",fontsize=20)
#         ax.set_ylabel("$g(r_{AD})$ ",fontsize=20)
#     elif type_=="AA":
#         ax.set_xlabel("$r_{AA}$ ($\AA$)",fontsize=20)
#         ax.set_ylabel("$g(r_{AA})$ ",fontsize=20)
#     elif type_=="DD":
#         ax.set_xlabel("$r_{DD}$ ($\AA$)",fontsize=20)
#         ax.set_ylabel("$g(r_{DD})$ ",fontsize=20)
#     elif type_=="DA":
#         ax.set_xlabel("$r_{DA}$ ($\AA$)",fontsize=20)
#         ax.set_ylabel("$g(r_{DA})$ ",fontsize=20)
        
#     ax.set_title(f"{bname}",fontsize=20)
    
#     plt.legend(fontsize=18)
#     plt.savefig(f"{bname}_{type_}.pdf")
#     plt.show()

In [ ]:
# comparison_strain(gr_m_0_ad_s,gr_m_50_ad_s,gr_m_100_ad_s,"PDKCM","AD")
comparison_strain(gr_m_0_aa_s,gr_m_50_aa_s,gr_m_100_aa_s,"PDKCM","AA")
comparison_strain(gr_m_0_dd_s,gr_m_50_dd_s,gr_m_100_dd_s,"PDKCM","DD")
# comparison_strain(gr_m_0_da_s,gr_m_50_da_s,gr_m_100_da_s,"PDKCM","DA")

In [ ]:
# comparison_strain(gr_p_0_ad_s,gr_p_50_ad_s,gr_p_100_ad_s,"PDKCP","AD")
comparison_strain(gr_p_0_aa_s,gr_p_50_aa_s,gr_p_100_aa_s,"PDKCP","AA")
comparison_strain(gr_p_0_dd_s,gr_p_50_dd_s,gr_p_100_dd_s,"PDKCP","DD")
# comparison_strain(gr_p_0_da_s,gr_p_50_da_s,gr_p_100_da_s,"PDKCP","DA")

In [ ]:
# comparison_strain(gr_h_0_ad_s,gr_h_50_ad_s,gr_h_100_ad_s,"PDKCH","AD")
comparison_strain(gr_h_0_aa_s,gr_h_50_aa_s,gr_h_100_aa_s,"PDKCH","AA")
comparison_strain(gr_h_0_dd_s,gr_h_50_dd_s,gr_h_100_dd_s,"PDKCH","DD")
# comparison_strain(gr_h_0_da_s,gr_h_50_da_s,gr_h_100_da_s,"PDKCH","DA")

In [ ]:
# comparison_strain(gr_d_0_ad_s,gr_d_50_ad_s,gr_d_100_ad_s,"PDKCD","AD")
comparison_strain(gr_d_0_aa_s,gr_d_50_aa_s,gr_d_100_aa_s,"PDKCD","AA")
comparison_strain(gr_d_0_dd_s,gr_d_50_dd_s,gr_d_100_dd_s,"PDKCD","DD")
# comparison_strain(gr_d_0_da_s,gr_d_50_da_s,gr_d_100_da_s,"PDKCD","DA")

In [ ]:
# def comparison_materials(info_m,info_p,info_h,info_d,type_):
#     fig = plt.figure(figsize=(8,6))
#     ax = fig.add_subplot(111)

#     ax.plot(info_m[:-10,0],info_m[:-10,1],color="red",label='PDKCM')
#     ax.plot(info_p[:-10,0],info_p[:-10,1],color="blue",label='PDKCP')
#     ax.plot(info_h[:-10,0],info_h[:-10,1],color="orange",label='PDKCH')
#     ax.plot(info_d[:-10,0],info_d[:-10,1],color="green",label='PDKCD')

#     if type_=="AD":
#         ax.set_xlabel("$r_{AD}$ ($\AA$)",fontsize=20)
#         ax.set_ylabel("$g(r_{AD})$ ",fontsize=20)
#     elif type_=="AA":
#         ax.set_xlabel("$r_{AA}$ ($\AA$)",fontsize=20)
#         ax.set_ylabel("$g(r_{AA})$ ",fontsize=20)
#     elif type_=="DD":
#         ax.set_xlabel("$r_{DD}$ ($\AA$)",fontsize=20)
#         ax.set_ylabel("$g(r_{DD})$ ",fontsize=20)
#     elif type_=="DA":
#         ax.set_xlabel("$r_{DA}$ ($\AA$)",fontsize=20)
#         ax.set_ylabel("$g(r_{DA})$ ",fontsize=20)

#     plt.legend(fontsize=18)
#     plt.savefig(f"All_{type_}.pdf")
#     plt.show()

In [ ]:
# comparison_materials(gr_m_0_ad_s,gr_p_0_ad_s,gr_h_0_ad_s,gr_d_0_ad_s,"AD")
# comparison_materials(gr_m_0_aa_s,gr_p_0_aa_s,gr_h_0_aa_s,gr_d_0_aa_s,"AA")
# comparison_materials(gr_m_0_dd_s,gr_p_0_dd_s,gr_h_0_dd_s,gr_d_0_dd_s,"DD")
# comparison_materials(gr_m_0_da_s,gr_p_0_da_s,gr_h_0_da_s,gr_d_0_da_s,"DA")

In [ ]:
def write_data(data_AA,data_DD,data_AA_smooth,data_DD_smooth,name,strain):
    
    def average(data):
        info_ = np.zeros((len(data[0]),2))
        
        for r in range (len(data[0])):
            info_[r,0] = data[0][r,0]
            for t in range (len(data)):
                info_[r,1] += data[t][r,1]/len(data)
        return info_
            
    info_aa = average(data_AA)
    info_dd = average(data_DD)
    info_aas = average(data_AA_smooth)
    info_dds = average(data_DD_smooth)
    
    file = open(f"{name}_{strain}.txt","w")
    file.write("distance      AA      DD      AA(s)      DD(s)")
    file.write("\n")
    
    for i in range (len(info_aa)):
        file.write("%.2f"%(info_aa[i,0])+"      ")
        file.write("%.4f"%(info_aa[i,1])+"     ")
        file.write("%.4f"%(info_dd[i,1])+"     ")
        file.write("%.4f"%(info_aas[i,1])+"     ")
        file.write("%.4f"%(info_dds[i,1])+"     ")
        file.write("\n")
    file.close()

In [ ]:
# # write_data(gr_m_0_aa,gr_m_0_dd,gr_m_0_aa_s,gr_m_0_dd_s,"M",0)
# write_data(gr_m_50_aa,gr_m_50_dd,gr_m_50_aa_s,gr_m_50_dd_s,"M",50)
# write_data(gr_m_100_aa,gr_m_100_dd,gr_m_100_aa_s,gr_m_100_dd_s,"M",100)

# # write_data(gr_p_0_aa,gr_p_0_dd,gr_p_0_aa_s,gr_p_0_dd_s,"P",0)
# write_data(gr_p_50_aa,gr_p_50_dd,gr_p_50_aa_s,gr_p_50_dd_s,"P",50)
# write_data(gr_p_100_aa,gr_p_100_dd,gr_p_100_aa_s,gr_p_100_dd_s,"P",100)

# # write_data(gr_h_0_aa,gr_h_0_dd,gr_h_0_aa_s,gr_h_0_dd_s,"H",0)
# write_data(gr_h_50_aa,gr_h_50_dd,gr_h_50_aa_s,gr_h_50_dd_s,"H",50)
# write_data(gr_h_100_aa,gr_h_100_dd,gr_h_100_aa_s,gr_h_100_dd_s,"H",100)

# # write_data(gr_d_0_aa,gr_d_0_dd,gr_d_0_aa_s,gr_d_0_dd_s,"D",0)
# write_data(gr_d_50_aa,gr_d_50_dd,gr_d_50_aa_s,gr_d_50_dd_s,"D",50)
# write_data(gr_d_100_aa,gr_d_100_dd,gr_d_100_aa_s,gr_d_100_dd_s,"D",100)

In [ ]:
def have_a_look(data_ori,data_smooth,name,type_):
    fig = plt.figure(figsize=(8,6))
    ax = fig.add_subplot(111)

    def average(data):
        info_ = np.zeros((len(data[0]),2))

        for r in range (len(data[0])):
            info_[r,0] = data[0][r,0]
            for t in range (len(data)):
                info_[r,1] += data[t][r,1]/len(data)
        return info_

    test1 = average(data_ori)
    test2 = average(data_smooth)

    ax.plot(test1[:-10,0],test1[:-10,1],color="black",label='ori')
    ax.plot(test2[:-10,0],test2[:-10,1],color="red",label='smooth')
    
    ax.set_xlabel("r ($\AA$)",fontsize=20)
    ax.set_ylabel("$g_{r}$",fontsize=20)
    ax.set_title(f"{name} {type_}",fontsize=20)
    
    plt.legend()
    plt.show()